In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

pd.options.display.max_columns = None

In [2]:
url = 'http://www.nhl.com/scores/htmlreports/20182019/PL020683.HTM'
response = requests.get(url)

# lxml parser required
soup = BeautifulSoup(response.text)

In [3]:
trs = soup.find_all('tr', class_='evenColor')

In [4]:
play_by_play_data = {
    'event_id' : [],
    'period' : [],
    'strength' : [],
    'time_elapsed' : [],
    'event_type' : [],
    'event' : [],
    'visitor_on_ice' : [],
    'home_on_ice' : []
}

key_lookup = {
    1 : 'event_id',
    3 : 'period',
    5 : 'strength',
    7 : 'time_elapsed',
    9 : 'event_type',
    11 : 'event',
    13 : 'visitor_on_ice',
    15 : 'home_on_ice'
}
pattern = r'\n+'

for row in trs[4:]:
    for i, cell in enumerate(row):
        
        if i in [1, 3, 5, 7, 9, 11, 13, 15]:
            text = cell.text.replace('\xa0', ' ')
            text = re.sub(pattern, '', text)
            
            key = key_lookup.get(i)
            play_by_play_data[key].append(text)

In [5]:
df = pd.DataFrame(play_by_play_data)

In [6]:
home_on_ice = ( df['home_on_ice']
                .str.strip()
                .str.replace('[aA-zZ]', '')
                .str.split(' ', expand=True) )
home_cols = ['home_1', 'home_2', 'home_3', 'home_4', 'home_5', 'home_6']
home_on_ice.columns = home_cols

visitor_on_ice = ( df['visitor_on_ice']
                   .str.strip()
                   .str.replace('[aA-zZ]', '')
                   .str.split(' ', expand=True) )
visitor_cols = ['visitor_1', 'visitor_2', 'visitor_3', 'visitor_4', 'visitor_5', 'visitor_6']
visitor_on_ice.columns = visitor_cols

In [7]:
df[home_cols] = home_on_ice
df[visitor_cols] = visitor_on_ice

In [8]:
df.head()

,event_id,period,strength,time_elapsed,event_type,event,visitor_on_ice,home_on_ice,home_1,home_2,home_3,home_4,home_5,home_6,visitor_1,visitor_2,visitor_3,visitor_4,visitor_5,visitor_6
0,5,1,EV,0:0020:00,FAC,BUF won Neu. Zone - BUF #9 EICHEL vs CAR #20 AHO,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,20,79,86,22,74,35,9,53,72,6,19,40
1,6,1,EV,0:2219:38,GIVE,"CAR GIVEAWAY - #86 TERAVAINEN, Def. Zone",9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,20,79,86,22,74,35,9,53,72,6,19,40
2,7,1,EV,0:2919:31,TAKE,"CAR TAKEAWAY - #20 AHO, Def. Zone",9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,20,79,86,22,74,35,9,53,72,6,19,40
3,8,1,EV,1:1718:43,HIT,"CAR #27 FAULK HIT BUF #43 SHEARY, Neu. Zone",17C 23C 43L 24D 55D 40G,71C 37R 48L 27D 44D 35G,71,37,48,27,44,35,17,23,43,24,55,40
4,9,1,EV,1:3218:28,GIVE,"CAR GIVEAWAY - #44 DE HAAN, Neu. Zone",71C 21R 22L 4D 26D 40G,42C 14R 23L 44D 57D 35G,42,14,23,44,57,35,71,21,22,4,26,40


We notice that the time elapsed and time remaining column are in one column, so we will separate them. Since each time follows the format n+:nn, we will split the column on the index that is 2 after the first colon.

In [9]:
times = df['time_elapsed'].str.split(':', expand=True)

times['time_elapsed'] = times[0] + ':' + times[1].str[:2]
times['time_remaining'] = times[1].str[2:] + ':' + times[2]

In [10]:
times.head(10)

,0,1,2,time_elapsed,time_remaining
0,0,0020,00,0:00,20:00
1,0,2219,38,0:22,19:38
2,0,2919,31,0:29,19:31
3,1,1718,43,1:17,18:43
4,1,3218,28,1:32,18:28
5,2,0817,52,2:08,17:52
6,2,1217,48,2:12,17:48
7,2,1317,47,2:13,17:47
8,2,1817,42,2:18,17:42
9,2,2917,31,2:29,17:31


Add the new columns back into the original DataFrame

In [11]:
df[['time_elapsed', 'time_remaining']] = times[['time_elapsed', 'time_remaining']]

In [12]:
col_order = [
    'event_id', 'period', 'strength', 'time_elapsed', 'time_remaining',
    'event_type', 'event', 'visitor_on_ice', 'home_on_ice',
    'home_1', 'home_2', 'home_3', 'home_4', 'home_5', 'home_6',
    'visitor_1', 'visitor_2', 'visitor_3', 'visitor_4', 'visitor_5', 'visitor_6'
]

df = df[col_order]

In [13]:
df.head()

,event_id,period,strength,time_elapsed,time_remaining,event_type,event,visitor_on_ice,home_on_ice,home_1,home_2,home_3,home_4,home_5,home_6,visitor_1,visitor_2,visitor_3,visitor_4,visitor_5,visitor_6
0,5,1,EV,0:00,20:00,FAC,BUF won Neu. Zone - BUF #9 EICHEL vs CAR #20 AHO,9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,20,79,86,22,74,35,9,53,72,6,19,40
1,6,1,EV,0:22,19:38,GIVE,"CAR GIVEAWAY - #86 TERAVAINEN, Def. Zone",9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,20,79,86,22,74,35,9,53,72,6,19,40
2,7,1,EV,0:29,19:31,TAKE,"CAR TAKEAWAY - #20 AHO, Def. Zone",9C 53C 72C 6D 19D 40G,20R 79L 86L 22D 74D 35G,20,79,86,22,74,35,9,53,72,6,19,40
3,8,1,EV,1:17,18:43,HIT,"CAR #27 FAULK HIT BUF #43 SHEARY, Neu. Zone",17C 23C 43L 24D 55D 40G,71C 37R 48L 27D 44D 35G,71,37,48,27,44,35,17,23,43,24,55,40
4,9,1,EV,1:32,18:28,GIVE,"CAR GIVEAWAY - #44 DE HAAN, Neu. Zone",71C 21R 22L 4D 26D 40G,42C 14R 23L 44D 57D 35G,42,14,23,44,57,35,71,21,22,4,26,40


We will now export this dataset to be used in our analysis specifically focused on faceoffs.

In [14]:
df.to_csv('on_ice.csv', index=False)